In [2]:
import pandas as pd
import duckdb
import altair as alt

In [21]:
# ¿Por que razones el voto no sumo a ningun partido?

no_positivos = duckdb.sql("""
    SELECT votos_tipo, SUM(votos_cantidad) AS votos 
    FROM read_parquet('ResultadosElectorales_*.parquet') 
    WHERE cargo_id = 1 AND votos_tipo != 'POSITIVO'
    GROUP BY 1
""").df()
no_positivos

chart = alt.Chart(no_positivos).mark_bar(color='steelblue').encode(
    y=alt.Y('votos_tipo:N', sort='-x', title=None),
    x='votos:Q'
).properties(
    title='Votos por Tipo',
    height=400
)

chart

alt.Chart(...)

In [19]:
# ¿Cuantos votos recibieron en las elecciones GENERALES los 5 patidos más relevantes?

agrupacion_1v = duckdb.sql("""
  SELECT agrupacion_nombre, sum(votos_cantidad) as votos
  FROM read_parquet('ResultadosElectorales_1v.parquet') 
  WHERE agrupacion_nombre IS NOT NULL AND cargo_id = 1
  GROUP BY agrupacion_nombre
  ORDER BY 2 desc
""").df()

agrupacion_1v['agrupacion_nombre'] = agrupacion_1v['agrupacion_nombre'].replace(
    {'FRENTE DE IZQUIERDA Y DE TRABAJADORES - UNIDAD': 'FRENTE DE IZQUIERDA'})
# Formatear los votos
agrupacion_1v['votos_formateados'] = agrupacion_1v['votos'].apply(lambda x: '{:,}'.format(x))
agrupacion_1v

colors_dict = {
    "JUNTOS POR EL CAMBIO": "#fedd00",
    "UNION POR LA PATRIA": "#009cde",
    "LA LIBERTAD AVANZA": "#753bbc",
    "FRENTE DE IZQUIERDA": "#f95461",
    "HACEMOS POR NUESTRO PAIS": "#43488f"
}

# Crear el gráfico
base = alt.Chart(agrupacion_1v).encode(
    x=alt.X('votos:Q', title="Votos"),
    y=alt.Y('agrupacion_nombre:N', sort='-x', title=None),
    color=alt.Color('agrupacion_nombre:N', scale=alt.Scale(domain=list(colors_dict.keys()), range=list(colors_dict.values())), legend=None),
)

# Configuración de las barras
bars = base.mark_bar().properties(
    title="Presidente, GENERALES 2023"
)

# Configuración del texto
text = base.mark_text(align='left', dx=2, color='black').encode(
    text=alt.Text('votos_formateados:N')
)

# Mostrar el gráfico
chart = (bars + text)
chart.configure_axis(labelFontSize=12, titleFontSize=14).configure_title(fontSize=16)

alt.LayerChart(...)

In [87]:
# ¿Cuanta gente voto desde el extranjero?

cant_extranjeros = duckdb.sql("""
    SELECT eleccion_tipo,mesa_tipo, SUM(votos_cantidad) AS votos 
    FROM read_parquet('ResultadosElectorales_*.parquet') 
    GROUP BY mesa_tipo,eleccion_tipo""").df()
cant_extranjeros

chosen_eleccion_tipo = 'GENERAL'

# Filter the data for the chosen eleccion_tipo
subset_data = cant_extranjeros[cant_extranjeros['eleccion_tipo'] == chosen_eleccion_tipo]

# Create a pie chart with text labels for each mesa_tipo
chart = alt.Chart(subset_data).mark_arc().encode(
    color='mesa_tipo:N',
    tooltip=['mesa_tipo:N', 'votos:Q'],
    angle='votos:Q'
).properties(
    title=f'Distribution of Votes for {chosen_eleccion_tipo} by mesa_tipo'
)

# Add text labels for each mesa_tipo with stacking
text = chart.mark_text(
    align='center',
    baseline='middle',
    dx=8,  
    radius=70, 
    color='black', 
    fontWeight='bold', 
    fontSize=14  
).encode(
    text='votos:Q',
    color=alt.value('black'),  
    angle='votos:Q'
)

# Combine the chart and text labels
final_chart = alt.layer(chart,text)

final_chart 

alt.LayerChart(...)